In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

# Создаем SparkSession и настраиваем количество партиций для shuffle
spark = SparkSession.builder \
    .appName("Shuffle Example") \
    .config("spark.sql.shuffle.partitions", "50") \
    .getOrCreate()

# Проверяем текущую настройку количества партиций для shuffle
print(f"Current shuffle partitions setting: {spark.conf.get('spark.sql.shuffle.partitions')}")

# Пример данных с большим объемом
data1 = [(i, f"Name_{i % 5}") for i in range(1000000)]
data2 = [(i, f"Category_{i % 3}") for i in range(1000000)]

df1 = spark.createDataFrame(data1, ["id", "name"])
df2 = spark.createDataFrame(data2, ["id", "category"])

# Проверяем начальное количество партиций
initial_partitions_df1 = df1.rdd.getNumPartitions()
initial_partitions_df2 = df2.rdd.getNumPartitions()
print(f"Initial number of partitions in df1: {initial_partitions_df1}")
print(f"Initial number of partitions in df2: {initial_partitions_df2}")

# Принудительно увеличиваем количество партиций перед join
df1_repartitioned = df1.repartition(50)
df2_repartitioned = df2.repartition(50)

# Проверяем количество партиций после repartition
repartitioned_partitions_df1 = df1_repartitioned.rdd.getNumPartitions()
repartitioned_partitions_df2 = df2_repartitioned.rdd.getNumPartitions()
print(f"Number of partitions in df1 after repartition: {repartitioned_partitions_df1}")
print(f"Number of partitions in df2 after repartition: {repartitioned_partitions_df2}")

# Выполняем операцию join, требующую shuffle
joined_df = df1_repartitioned.join(df2_repartitioned, "id")

# Проверяем количество партиций после join
joined_partitions = joined_df.rdd.getNumPartitions()
print(f"Number of partitions after join: {joined_partitions}")

# Останавливаем SparkSession
spark.stop()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/07/30 10:07:37 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/07/30 10:07:40 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


Current shuffle partitions setting: 50
